In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('flattened_data.csv')

C:\Users\maxma\AppData\Local\Temp\ipykernel_4292\1381573417.py:1: DtypeWarning: Columns (11,12,14,15,21,22,23,24,25,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('flattened_data.csv')


In [3]:
df

,id,tripUpdate.trip.tripId,tripUpdate.trip.startTime,tripUpdate.trip.startDate,tripUpdate.trip.routeId,tripUpdate.trip.directionId,tripUpdate.vehicle.id,tripUpdate.timestamp,tripUpdate.vehicle.label,tripUpdate.trip.scheduleRelationship,...,vehicle.currentStatus,vehicle.timestamp,vehicle.stopId,vehicle.vehicle.id,vehicle.vehicle.label,vehicle.occupancyStatus,vehicle.occupancyPercentage,vehicle.multiCarriageDetails,vehicle.position.speed,first_stop_time
0,63100735,63100735,14:30:00,20240815.0,100,0.0,y1404,2024-08-15 18:20:51,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-15 18:30:00
1,63473633,63473633,15:10:00,20240815.0,441442,0.0,y3356,2024-08-15 18:20:52,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-15 19:10:00
2,63071440,63071440,14:38:00,20240815.0,17,1.0,y1830,2024-08-15 18:20:55,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-15 18:38:00
3,63480392,63480392,14:20:00,20240815.0,28,0.0,y1250,2024-08-15 18:20:53,1250,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-15 18:21:09
4,63288413,63288413,14:44:00,20240815.0,Red,0.0,R-547DFF8B,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-15 18:50:14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51448,y1875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,STOPPED_AT,2024-08-15 18:50:51,1514,y1875,1875,MANY_SEATS_AVAILABLE,20.0,NaN,NaN,NaN
51449,y1927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,IN_TRANSIT_TO,2024-08-15 18:50:52,73,y1927,1927,MANY_SEATS_AVAILABLE,20.0,NaN,NaN,NaN
51450,y2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,IN_TRANSIT_TO,2024-08-15 18:50:49,5695,y2018,2018,MANY_SEATS_AVAILABLE,0.0,NaN,NaN,NaN
51451,y1682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,STOPPED_AT,2024-08-15 18:50:52,37618,y1682,1682,MANY_SEATS_AVAILABLE,0.0,NaN,NaN,NaN


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report

# Load the data with specified data types
data_iterator = pd.read_csv(
    'flattened_data.csv',
    chunksize=100000,  # Adjust chunk size based on your system's memory
    low_memory=False,  # Handle mixed type columns
    dtype={
        'vehicle.position.latitude': 'float64',
        'vehicle.position.longitude': 'float64',
        'vehicle.timestamp': 'str',  # Keep timestamp as string to convert later
        'tripUpdate.trip.tripId': 'str'  # Assuming tripId should be treated as a category
    }
)

# Initialize an empty dataframe to hold all processed data
all_data = pd.DataFrame()

# Function to preprocess each chunk
def preprocess_chunk(df):
    # Convert the 'vehicle.timestamp' to datetime
    df['vehicle.timestamp'] = pd.to_datetime(df['vehicle.timestamp'])
    
    # Feature engineering from the timestamp
    df['hour'] = df['vehicle.timestamp'].dt.hour
    df['minute'] = df['vehicle.timestamp'].dt.minute
    df['day_of_week'] = df['vehicle.timestamp'].dt.dayofweek
    
    # Return the processed dataframe
    return df

# Process each chunk
for chunk in data_iterator:
    processed_chunk = preprocess_chunk(chunk)
    all_data = pd.concat([all_data, processed_chunk], ignore_index=True)

# Define features and labels
X = all_data[['vehicle.position.latitude', 'vehicle.position.longitude', 'hour', 'minute', 'day_of_week']]
y = all_data['tripUpdate.trip.tripId']

# Impute missing values
imputer = SimpleImputer(strategy='mean')  # You can change the strategy
X = imputer.fit_transform(X)

# Check if the target variable has missing values
y = y.fillna('Unknown')  # You can choose to drop NaNs or assign a placeholder value

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train a model (e.g., Random Forest)
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.3162957924399961
                                             precision    recall  f1-score   support

                                   62860944       0.00      0.00      0.00         1
                                   62860945       0.00      0.00      0.00         2
                                   62860946       0.00      0.00      0.00         4
                                   62860947       0.00      0.00      0.00         2
                                   62860986       0.00      0.00      0.00         2
                                   62860987       0.00      0.00      0.00         5
                                   62860988       0.00      0.00      0.00         7
                                   62860989       0.00      0.00      0.00         1
                                   62860990       0.00      0.00      0.00         3
                                   62860991       0.00      0.00      0.00         3
                                   

C:\Users\maxma\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\maxma\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\maxma\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Target variable being predicted is tripUpdate.trip.tripId, which represents the Trip ID.**

Features (Input Variables):
- vehicle.position.latitude: The latitude coordinate of the vehicle.
- vehicle.position.longitude: The longitude coordinate of the vehicle.
- hour: The hour of the day, extracted from the vehicle.timestamp.
- minute: The minute of the hour, also extracted from the vehicle.timestamp.
- day_of_week: The day of the week (0 for Monday, 6 for Sunday), extracted from the vehicle.timestamp.

In [5]:
# pip install dependencies not already installed

! pip install xgboost
! pip install lightgbm
! pip install catboost

In [6]:
# Import necessary libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


In [7]:
# Assuming X_train, X_test, y_train, y_test are already defined as:
# Train-test split (assuming 'X' are features and 'y' is the target variable)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features (if needed)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [8]:
# Train-test split (assuming 'X' are features and 'y' is the target variable)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Impute missing values if necessary (already done in original code)
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features (if needed)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [10]:
# Train a model (e.g., Random Forest)
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.3162957924399961


C:\Users\maxma\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\maxma\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                             precision    recall  f1-score   support

                                   62860944       0.00      0.00      0.00         1
                                   62860945       0.00      0.00      0.00         2
                                   62860946       0.00      0.00      0.00         4
                                   62860947       0.00      0.00      0.00         2
                                   62860986       0.00      0.00      0.00         2
                                   62860987       0.00      0.00      0.00         5
                                   62860988       0.00      0.00      0.00         7
                                   62860989       0.00      0.00      0.00         1
                                   62860990       0.00      0.00      0.00         3
                                   62860991       0.00      0.00      0.00         3
                                   62860993       0.00      0.00

C:\Users\maxma\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# LOGISTIC REGRESSION

# Train and evaluate Logistic Regression
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Logistic Regression Accuracy: {accuracy:.4f}")

# Get classification report with overall metrics
report = classification_report(y_test, y_pred, output_dict=True)

# Print only overall metrics (accuracy, macro avg, and weighted avg)
print(f"Precision (macro avg): {report['macro avg']['precision']:.4f}")
print(f"Recall (macro avg): {report['macro avg']['recall']:.4f}")
print(f"F1-Score (macro avg): {report['macro avg']['f1-score']:.4f}")
print(f"Precision (weighted avg): {report['weighted avg']['precision']:.4f}")
print(f"Recall (weighted avg): {report['weighted avg']['recall']:.4f}")
print(f"F1-Score (weighted avg): {report['weighted avg']['f1-score']:.4f}")

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', square=True)
plt.title("Confusion Matrix for Logistic Regression")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


In [ ]:
# SUPPORT VECTOR MACHINE (SVM)
# Train and evaluate SVM
model = SVC(kernel='rbf', random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Calculate accuracy and print metrics
accuracy = accuracy_score(y_test, y_pred)
print(f"SVM (RBF Kernel) Accuracy: {accuracy:.4f}")
print(f"Classification Report for SVM:\n", classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', square=True)
plt.title("Confusion Matrix for SVM")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


In [ ]:
# K-NEAREST NEIGHBORS (KNN)
# Train and evaluate K-Nearest Neighbors
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Calculate accuracy and print metrics
accuracy = accuracy_score(y_test, y_pred)
print(f"K-Nearest Neighbors Accuracy: {accuracy:.4f}")
print(f"Classification Report for K-Nearest Neighbors:\n", classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', square=True)
plt.title("Confusion Matrix for K-Nearest Neighbors")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


In [ ]:
# GRADIENT BOOSTING
# Train and evaluate Gradient Boosting
model = GradientBoostingClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Calculate accuracy and print metrics
accuracy = accuracy_score(y_test, y_pred)
print(f"Gradient Boosting Accuracy: {accuracy:.4f}")
print(f"Classification Report for Gradient Boosting:\n", classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', square=True)
plt.title("Confusion Matrix for Gradient Boosting")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


In [ ]:
# XGBOOST
# Train and evaluate XGBoost
model = XGBClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Calculate accuracy and print metrics
accuracy = accuracy_score(y_test, y_pred)
print(f"XGBoost Accuracy: {accuracy:.4f}")
print(f"Classification Report for XGBoost:\n", classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', square=True)
plt.title("Confusion Matrix for XGBoost")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


In [ ]:
# MLP (NEURAL NETWORK)
# Train and evaluate MLP (Neural Network)
model = MLPClassifier(hidden_layer_sizes=(100, 50), random_state=42, max_iter=500)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Calculate accuracy and print metrics
accuracy = accuracy_score(y_test, y_pred)
print(f"MLP (Neural Network) Accuracy: {accuracy:.4f}")
print(f"Classification Report for MLP:\n", classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', square=True)
plt.title("Confusion Matrix for MLP")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


In [ ]:
# LIGHTGBM
# Train and evaluate LightGBM
model = LGBMClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Calculate accuracy and print metrics
accuracy = accuracy_score(y_test, y_pred)
print(f"LightGBM Accuracy: {accuracy:.4f}")
print(f"Classification Report for LightGBM:\n", classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', square=True)
plt.title("Confusion Matrix for LightGBM")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


In [ ]:
# CATBOOST
# Train and evaluate CatBoost
model = CatBoostClassifier(random_state=42, verbose=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Calculate accuracy and print metrics
accuracy = accuracy_score(y_test, y_pred)
print(f"CatBoost Accuracy: {accuracy:.4f}")
print(f"Classification Report for CatBoost:\n", classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', square=True)
plt.title("Confusion Matrix for CatBoost")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

